### Initiation

In [58]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [59]:
# read the words
words = open('names.txt', 'r').read().splitlines()
len(words), words[:5]

(32033, ['emma', 'olivia', 'ava', 'isabella', 'sophia'])

In [60]:
# build the vocabulary
chars = sorted(list(set(''.join(words))))
s_to_i = {'.' : 0} | {s: i + 1 for i, s in enumerate(chars)}
i_to_s = {i: s for s, i in s_to_i.items()}
vocab_size = len(i_to_s)
print(i_to_s)
print(vocab_size)

{0: '.', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
27


In [61]:
# build the dataset

block_size = 3 # context length (how many chars to predict next)

def build_dataset(words):
	X, Y = [], []

	for w in words:
		context = [0] * block_size
		for ch in w + '.':
			idx = s_to_i[ch]
			X.append(context)
			Y.append(idx)
			context = context[1:] + [idx]
	
	X = torch.tensor(X)
	Y = torch.tensor(Y)
	print(X.shape, Y.shape)
	return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(len(words) * 0.8)
n2 = int(len(words) * 0.9)

X_train,	Y_train	= build_dataset(words[:n1])		# 80%
X_val,		Y_val	= build_dataset(words[n1:n2])	# 10%
X_test,		Y_test	= build_dataset(words[n2:])		# 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [62]:
# util function used for comparing manual gradients to PyTorch's autograd
def cmp(s, dt, t):
	ex = torch.all(dt == t.grad).item()
	app = torch.allclose(dt, t.grad)
	maxdiff = (dt - t.grad).abs().max().item()
	pad = 1 if app else 10
	print(f'{s:15s} | exact: {str(ex):5s} | approx: {str(app):5s} | max diff: {maxdiff:.{pad}f}')

In [63]:
n_embd = 10 # embedding size
n_hidden = 64 # hidden layer size

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd),				generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden),	generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn((n_hidden,),						generator=g) * 0.1 # for fun (useless)
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),			generator=g) * 0.1
b2 = torch.randn((vocab_size,),						generator=g) * 0.1

# BatchNorm parameters
bn_gain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bn_bias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
	p.requires_grad = True

4137


In [64]:
batch_size = 32
n = batch_size # a short var for convenience
# construct minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
Xb, Yb = X_train[ix], Y_train[ix] # batch X, Y

In [65]:
# forward pass that's "chunkated" into smaller steps to backward one at a time

emb = C[Xb] # embed the chars into vectors
embcat = emb.view(emb.shape[0], -1) # concat the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer preactivation

# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * bndiff2.sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bn_gain * bnraw + bn_bias

# Non-linearity
h = torch.tanh(hpreact) # hidden layer activation

# Linear layer 2
logits = h @ W2 + b2 # output layer

# cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
	p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, h, hpreact, bnraw, bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani, embcat, emb]:
	t.retain_grad()
loss.backward()
loss

tensor(3.3184, grad_fn=<NegBackward0>)

In [66]:
logprobs.shape, logprobs[range(n), Yb], Yb.shape

(torch.Size([32, 27]),
 tensor([-4.0407, -3.0782, -3.5118, -3.2453, -4.0622, -3.5889, -3.1562, -3.9936,
         -3.2445, -4.2271, -3.2056, -1.6667, -2.9298, -2.9671, -2.8364, -3.0070,
         -3.8745, -2.8850, -3.6881, -3.2255, -2.7731, -2.8339, -4.2519, -3.9529,
         -3.5456, -2.9495, -2.8694, -3.8788, -2.8491, -3.4116, -3.1982, -3.2408],
        grad_fn=<IndexBackward0>),
 torch.Size([32]))

In [68]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n

dprobs = (1.0 / probs) * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
dnorm_logits = counts * dcounts
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)

logprobs        | exact: True  | approx: True  | max diff: 0.0
probs           | exact: True  | approx: True  | max diff: 0.0
counts_sum_inv  | exact: True  | approx: True  | max diff: 0.0
counts_sum      | exact: True  | approx: True  | max diff: 0.0
counts          | exact: True  | approx: True  | max diff: 0.0
norm_logits     | exact: True  | approx: True  | max diff: 0.0
logit_maxes     | exact: True  | approx: True  | max diff: 0.0
logits          | exact: False | approx: False | max diff: 0.0000000922
